In [20]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import wave
import IPython.display as ipd # load a local WAV file and PLAY!!!
import librosa
import librosa.display
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from scipy.signal import butter , filtfilt
import glob
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [2]:
set_a = pd.read_csv('inputsound/set_a.csv')
set_b = pd.read_csv('inputsound/set_b.csv')
set_a_timing = pd.read_csv('inputsound/set_a_timing.csv')
train_ab = pd.concat([set_a, set_b])

In [3]:
# GET THE NAME OF THE FILES IN A PATH
def get_files(path):
    for file in glob.glob(path + "/*"):
        print(file)

#get_data("inputsound/set_a")
#example: inputsound/set_a\artifact__201012172012.wav

# Low pass filter

*Low-frequency sounds are those whose dominant frequencies are less than 100 Hz, such as third and fourth heart sounds and the diastolic murmur of mitral stenosis. The murmur containing the highest frequency sound is aortic regurgitation, whose dominant frequencies are approximately 400 Hz. The principal frequencies of other sounds and murmurs are between 1 and 300 Hz.*

![Heart frequencies](images/heart_freq.jpg)

**FUNCTION TO GET THE AUDIOS FILTERED**
* signals = list where each index is an audiowhich has been filtered
* clases = list where each index is the class of the audio

In [18]:
def get_signals_filtered(path):
    signals,classes = [],[]
    # LPH, fc = 500Hz
    fc = 500
    sr = 22050
    w = fc/(sr/2)
    b,a = butter(3,w , btype = "lowpass")

    for file in glob.glob(path + "/*" ):
        signal, sr = librosa.load(file, sr = 22050, res_type='kaiser_fast')
        x = filtfilt(b,a, signal)
        x = x/x.max()
        signals.append(x)
        classes.append(file.split("\\")[-1].split("_")[0])
    
    return signals,classes

signals,classes = get_signals_filtered("inputsound/set_a")

In [19]:
# Example: filtered audio
print(classes[150])
ipd.Audio(signals[150], rate = 22050)

normal


# LPF + FFT

**FUNCTION GET_DATA**
* X list, contains the signal of each file
* X array, puts all the signal into one only array

* y list, is a unique  list with the class of each file
* y array, contains the class of each file as a matrix

In [49]:
def lpf_fft(path):
    X,y = [],[]
    
    # LPH, fc = 500Hz
    fc = 500
    sr = 22050
    w = fc/(sr/2)
    b,a = butter(3,w , btype = "lowpass")
    
    for file in glob.glob(path + "/*" ):
        #load signal
        signal, sr = librosa.load(file, sr = sr, res_type='kaiser_fast')
        #LPF
        x = filtfilt(b,a, signal)
        x = x/x.max()
        # FFT
        fft = np.fft.fft(x, n = 50)
        
        X.append(fft)
        y.append(file.split("\\")[-1].split("_")[0])
        
    X = np.concatenate(X)
    y = np.array(y).reshape(-1,1)
    
    return X,y

In [50]:
Xa,ya = lpf_fft("inputsound/set_a")
Xb,yb = lpf_fft("inputsound/set_b")

In [51]:
X = np.concatenate([Xa,Xb])
y = np.concatenate([ya,yb])

In [52]:
np.unique(y)

array(['Aunlabelledtest', 'Bunlabelledtest', 'artifact', 'extrahls',
       'extrastole', 'murmur', 'normal'], dtype='<U15')

In [53]:
X.shape

(41600,)

In [54]:
y.shape

(832, 1)

# LPF + MFCC

# First Neural Network

In [57]:
# Some train_test_splitting
X_train,X_test,y_train,y_test=train_test_split(X,y)

ValueError: Found input variables with inconsistent numbers of samples: [41600, 832]

In [ ]:
# prepare input data
def prepare_inputs(X_train, X_test):
	ohe = OneHotEncoder()
	ohe.fit(X_train)
	X_train_enc = ohe.transform(X_train)
	X_test_enc = ohe.transform(X_test)
	return X_train_enc, X_test_enc
 
# prepare target
def prepare_targets(y_train, y_test):
	le = LabelEncoder()
	le.fit(y_train)
	y_train_enc = le.transform(y_train)
	y_test_enc = le.transform(y_test)
	return y_train_enc, y_test_enc
 

In [42]:
# Now, our NN!

# The keras base for sequential models
model = Sequential()

# The hidden layer
# We input the numer of nodes, the input size as this the first layer and the activation function
model.add(Dense(2,input_dim=1,activation="softplus"))

# Then the output layer
# Since we will not put an activation function, it shall be a simple sum.
# We must do this to get back to 1 node
model.add(Dense(1))

In [43]:
# Now we compile!
# We just must set the loss function and the optimizer.

opt = Adam(learning_rate=0.1)
model.compile(loss="mean_squared_error", optimizer=opt)

In [44]:
# And we train!
# We must define the number of epochs ( How many times we will go through the data (and backpropagate))
history = model.fit(X, onehotlabels, epochs=500, verbose=2)

Epoch 1/500


ValueError: in user code:

    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorf

In [ ]:

# And we can print our green squigle.
x = np.linspace(0,1,1000)
x = np.expand_dims(x,-1)
y_pred = model.predict(x)

In [ ]:

plt.grid()
plt.scatter(X,y)
plt.ylabel("Efficiency")
plt.xlabel("Dosage")
plt.plot(x,y_pred, c="green");

In [46]:
# Same Architecture
model = Sequential()
#model.add(Dense(2,input_dim=1,activation="softplus"))
model.add(Dense(4,input_dim=1,activation="softplus"))
#model.add(Dense(6,input_dim=1,activation="softplus"))
model.add(Dense(8,input_dim=1,activation="softplus"))
model.add(Dense(4,input_dim=1,activation="softplus"))
model.add(Dense(1))

In [47]:

opt = Adam(learning_rate=0.1)
model.compile(loss="mean_squared_error", optimizer=opt)

In [48]:

history = model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100


ValueError: in user code:

    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\UX533F\anaconda3\envs\tf\lib\site-packages\tensorf

In [ ]:

x = np.linspace(0,1,1000)
x = np.expand_dims(x,-1)
y_pred = model.predict(x)

In [ ]:
plt.grid()
plt.scatter(X,y)
plt.ylabel("Efficiency")
plt.xlabel("Dosage")
plt.plot(x,y_pred, c="green");

In [ ]:
plt.grid()
plt.plot(history.history["loss"])
plt.ylabel("Loss")
plt.xlabel("Epochs");

In [12]:
train_ab

,dataset,fname,label,sublabel
0,a,set_a/artifact__201012172012.wav,artifact,NaN
1,a,set_a/artifact__201105040918.wav,artifact,NaN
2,a,set_a/artifact__201105041959.wav,artifact,NaN
3,a,set_a/artifact__201105051017.wav,artifact,NaN
4,a,set_a/artifact__201105060108.wav,artifact,NaN
...,...,...,...,...
651,b,set_b/Btraining_normal_Btraining_noisynormal_2...,normal,noisynormal
652,b,set_b/Btraining_normal_Btraining_noisynormal_2...,normal,noisynormal
653,b,set_b/Btraining_normal_Btraining_noisynormal_2...,normal,noisynormal
654,b,set_b/Btraining_normal_Btraining_noisynormal_2...,normal,noisynormal


In [ ]:
# FUNCION DE FELIPE 
def get_data(path):
    X,y = [],[]
    for file in glob.glob(path + "/*" ):
        signal, sr = librosa.load(file, sr = 22050, res_type='kaiser_fast')
        X.append(signal)
        y.append(file.split("\\")[-1].split("_")[0])
        
    X = np.concatenate(X)
    y = np.array(y).reshape(-1,1)
    
    return X,y

X,y = get_data("inputsound/set_a")